Install library for QC

In [481]:
%pip install dwave-ocean-sdk
%pip install dwave-neal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Подключим все необходимые библиотеки

In [1]:
import random
import math
import itertools
import numpy as np

import dimod
from dimod import BinaryQuadraticModel
from dimod.reference.samplers import ExactSolver
from neal import SimulatedAnnealingSampler
from qdeepsdk import QDeepHybridSolver

Сгенерируем случайные данные

In [2]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def distance(p1: Point, p2: Point) -> int:
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

class TestCase:
    def __init__(self):
        self.ClientsNum = 0
        self.DepotsNum = 0
        self.CarsNum = 0
        self.ClientPositions = []
        self.DepotPositions = []
        self.CarDepot = []
        self.CarDepotMap = {}

    def DistClients(self, i: int, j: int) -> int:
        return distance(self.ClientPositions[i], self.ClientPositions[j])
    
    def DistDepotClient(self, d: int, c: int) -> int:
        return distance(self.DepotPositions[d], self.ClientPositions[c])

def GenerateTestCase(clientsNum: int, depotsNum: int, carsNum: int, maxPos: int)-> TestCase:

    if carsNum < depotsNum:
        raise Exception("Cars number is less than depots number")

    res = TestCase()
    res.ClientsNum = clientsNum
    res.DepotsNum = depotsNum
    res.CarsNum = carsNum
    res.CarDepot = [[0 for _ in range(depotsNum)] for _ in range(carsNum)]

    for i in range(clientsNum):
        res.ClientPositions.append(Point(random.randint(0, maxPos), random.randint(0, maxPos)))

    for i in range(depotsNum):
        res.DepotPositions.append(Point(random.randint(0, maxPos), random.randint(0, maxPos)))

    for i in range(depotsNum):
        res.CarDepot[i][i] = 1
        res.CarDepotMap[i] = i

    for i in range(depotsNum, carsNum):
        depot = random.randint(0, depotsNum - 1)
        res.CarDepot[i][depot] = 1
        res.CarDepotMap[i] = depot

    return res

Напишем функцию для раскрытия выражения вида $(a_1 + a_2 + ... + a_n)^2$, потому что почти все ограничения к этому сводятся

In [3]:
# Input [(1, "x1"), (-2, "x3")]
# Output ({"x1": 1, "x3": 4}, {("x1", "x3"): -4})
def square(vals: list)-> tuple:

    res = ({}, {})

    for val in vals:
        if val[1] != "":
            if val[1] not in res[0].keys():
                res[0][val[1]] = 0
            res[0][val[1]] += val[0] * val[0]
        
    for v1 in vals:
        for v2 in vals:
            if v1[1] == v2[1]:
                continue
            if v1[1] == "" and v2[1] == "":
                continue

            if v1[1] != "" and v2[1] != "":
                if (v1[1], v2[1]) not in res[1].keys():
                    res[1][(v1[1], v2[1])] = 0
                res[1][(v1[1], v2[1])] += v1[0] * v2[0]
            else:
                if v1[1]+v2[1] not in res[0].keys():
                    res[0][v1[1]+v2[1]] = 0
                res[0][v1[1]+v2[1]] += v1[0] * v2[0]

    return res

print(square([(1, "x1"), (-2, "x3"), (1, "")]))

({'x1': 3, 'x3': 0}, {('x1', 'x3'): -2, ('x3', 'x1'): -2})


Создадим класс модели QUBO:

In [14]:
class QUBOModel:
    def __init__(self):
        self.h = {}
        self.J = {}
        self.BigConstant = 1000000
        self.id = 0 # serail key for constructing values in inequalities

    def AddCoef(self, coefs: tuple):
        for key in coefs[0]:
            if key not in self.h.keys():
                self.h[key] = 0
            self.h[key] += coefs[0][key]
        
        for key in coefs[1]:
            if key not in self.J.keys():
                self.J[key] = 0
            self.J[key] += coefs[1][key]

    def PrepareCoefs(self):
        for k1 in self.h.keys():
            for k2 in self.h.keys():
                if k1 != k2 and (k1, k2) in self.J.keys() and (k2, k1) in self.J.keys():
                    self.J[(k1, k2)] += self.J[(k2, k1)]
                    del self.J[(k2, k1)]

    def GetLists(self)-> tuple:
        self.PrepareCoefs()
        return (self.h, self.J)

    def GetMatrix(self)-> tuple: # matrix (QUBO) + list (pos->argname) + map (argname->pos)
        self.PrepareCoefs()
        NamesArr = self.h.keys()
        NamesMap = {}
        for i in range(len(NamesArr)):
            NamesMap[NamesArr[i]] = i

        Matrix = np.zeros((len(NamesArr), len(NamesArr)), dtype=int)

        for key in self.h:
            Matrix[NamesMap[key]][NamesMap[key]] = self.h[key]

        for key in self.J:
            Matrix[NamesMap[key[0]]][NamesMap[key[1]]] = self.J[key]

        return (Matrix, NamesArr, NamesMap)
    
qubo = QUBOModel()

_Целевая функция:_ Функия описывающая издержки для передвижения всех машин. Данную функцию мы хотим минимизировать.

In [6]:
def min_func(test: TestCase, model: QUBOModel):
    for k in range(test.CarsNum):
        for i in range(test.ClientsNum):
            for j in range(test.ClientsNum):
                model.AddCoef(({test.DistClients(i, j), f"x{i}{j}{k}"}, {}))

    for k in range(test.CarsNum):
        for i in range (test.ClientsNum):
            for d in range(test.DepotsNum):
                model.AddCoef(({test.DistDepotClient(d, i)*test.CarDepot[k][d], f"u{i}{k}"}, {}))

    for k in range(test.CarsNum):
        for i in range (test.ClientsNum):
            for d in range(test.DepotsNum):
                model.AddCoef(({test.DistDepotClient(d, i)*test.CarDepot[k][d], f"n{i}{k}"}, {}))

_Ограничение 1:_ Нельзя поехать к тому же покупателю.

In [7]:
def constrint_1(test: TestCase, model: QUBOModel):
    for i in range(test.ClientsNum):
        for k in range(test.CarsNum):
            model.AddCoef(({model.BigConstant, f"x{i}{i}{k}"}, {}))

_Ограничение 2:_ К каждому покупателю должна приехать ровно одна машина.

In [8]:
def constraint_2(test: TestCase, model: QUBOModel):
    for i in range(test.ClientsNum):

        vals = []
        vals.append((model.BigConstant, ""))

        for k in range(test.CarsNum):
            for j in range(test.ClientsNum):
                vals.append((-model.BigConstant, f"x{j}{i}{k}"))
            vals.append((-model.BigConstant, f"u{i}{k}"))

        model.AddCoef(square(vals))

_Ограничение 3:_ От каждого покупателя должна уехать ровно одна машина.

In [9]:
def constraint_3(test: TestCase, model: QUBOModel):
    for i in range(test.ClientsNum):

        vals = []

        vals.append((model.BigConstant, ""))

        for k in range(test.CarsNum):
            for j in range(test.ClientsNum):
                vals.append((-model.BigConstant, f"x{i}{j}{k}"))
            vals.append((-model.BigConstant, f"n{i}{k}"))

        model.AddCoef(square(vals))

_Ограничение 4:_ Каждая машина должна отъехать ровно с одного склада.

In [11]:
def constraint_4(test: TestCase, model: QUBOModel):
    for k in range(test.CarsNum):

        vals = []

        vals.append((model.BigConstant, ""))

        for i in range(test.ClientsNum):
            vals.append((-model.BigConstant, f"u{i}{k}"))

        model.AddCoef(square(vals))

_Ограничение 5:_ Каждая машина должна приехать ровно на один склад.

In [12]:
def constraint_5(test: TestCase, model: QUBOModel):
    for k in range(test.CarsNum):

        vals = []

        vals.append((model.BigConstant, ""))

        for i in range(test.ClientsNum):
            vals.append((-model.BigConstant, f"n{i}{k}"))

        model.AddCoef(square(vals))

_Ограничение 6:_ "Целостность маршрута". Если машина приехала к покупателю, то она должна уехать от него.

In [13]:
def constraint_6(test: TestCase, model: QUBOModel):
    for i in range(test.ClientsNum):
        for k in range(test.CarsNum):
            
            vals = []

            vals.append((test.BigConstant, f"u{i}{k}"))
            vals.append((-test.BigConstant, f"n{i}{k}"))

            for j in range(test.ClientsNum):
                vals.append((test.BigConstant, f"x{j}{i}{k}"))
                vals.append((-test.BigConstant, f"x{i}{j}{k}"))

            model.AddCoef(square(vals))

_Ограничение 7:_ У машин не должно быть возможности иметь на своем маршруте цикл из городов, который они не посещают.

In [15]:
def get_subsets(n):
    res = []
    for i in range(1 << n):
        cur = []
        for j in range(n):
            if (i & (1 << j)):
                cur.append(j)
        res.append(cur)
    return res

def constraint_7(test: TestCase, model: QUBOModel):
    for S in get_subsets(test.ClientsNum):
        if len(S) >= 2:

            vals = []

            vals.append((model.BigConstant * (-len(S) + 1), ""))

            for k in range(test.CarsNum):
                for i in S:
                    for j in S:
                        if i != j:
                            vals.append((model.BigConstant, f"x{i}{j}{k}"))

            for l in range(math.ceil(math.log2(len(S)))):
                
                model.id += 1
                model.h[f"l{model.id}"] = 0
                vals.append((2**l * model.BigConstant, f"l{model.id}"))

            model.AddCoef(square(vals))

_Ограничение 10:_ Аналогичное ограничению 7, но работающее быстрее :)

In [ ]:
def constraint_10(test: TestCase, model: QUBOModel):
    log_t = math.ceil(math.log2(test.ClientsNum + 1))

    for i in range(test.ClientsNum):
        for j in range(test.ClientsNum):

            vals = []
            vals.append((model.BigConstant * (-test.ClientsNum + 1), ""))
            

Запустим квантовый отжиг

In [495]:
bqm = BinaryQuadraticModel(h, J, 'BINARY')

min_energy = 0
answer = {}

# Exact solver


# solution = ExactSolver().sample(bqm)
# print("ANSWER:", solution.first.energy)
# print(solution.first.sample)

# min_energy = solution.first.energy
# answer = solution.first.sample


# End Exact solver

for _ in range(10000):
    sample = SimulatedAnnealingSampler().sample(bqm, num_reads=1)
    if sample.first.energy < min_energy:
        min_energy = sample.first.energy
        answer = sample.first.sample

print("Min energy:", min_energy)
print("Optimizing parameters amount:", len(h.keys()))

total_distance = 0

for k in range(CarsNum):

    for i in range(ClientsNum):
        if answer[f"u{i}{k}"] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: Depot -> {i}, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")

    for i in range(ClientsNum):
        for j in range(ClientsNum):
            if answer[f"x{i}{j}{k}"] == 1:
                total_distance += Distances[i][j]
                print(f"Car {k}: {i} -> {j}, distance: {Distances[i][j]}")

    for i in range(ClientsNum):
        if answer[f"n{i}{k}"] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: {i} -> Depot, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")


print("Answer: ", total_distance)

Min energy: -366999999990298.0
Optimizing parameters amount: 202
Car 0: Depot -> 5, distance: 1047
Car 0: 1 -> 2, distance: 3447
Car 0: 3 -> 4, distance: 1783
Car 0: 4 -> 1, distance: 652
Car 0: 5 -> 3, distance: 1123
Car 0: 2 -> Depot, distance: 327
Car 1: Depot -> 0, distance: 665
Car 1: 0 -> Depot, distance: 665
Answer:  9709


Давайте вычислим ответ полным перебором

In [496]:
answer_complete_search = BigConstant

for perm in itertools.permutations(list(range(ClientsNum))):
    dist = Distances[perm[0]][ClientsNum] + Distances[perm[-1]][ClientsNum]
    for i in range(len(perm) - 1):
        dist += Distances[perm[i]][perm[i + 1]]

    answer_complete_search = min(answer_complete_search, dist)

print("Answer Complete Search:", answer_complete_search)

Answer Complete Search: 10579


Решим задачу используя QDeepSDK

In [ ]:
solver = QDeepHybridSolver()
solver.m_budget = 1000
solver.num_reads = 5000
solver.token = "hf6si03meu"

keys_array = list(h.keys())
keys_map = {}
for i in range(len(keys_array)):
    keys_map[keys_array[i]] = i

matrix = np.zeros((len(keys_array), len(keys_array)), dtype=int)

for key in h:
    matrix[keys_map[key]][keys_map[key]] = h[key]

for key in J:
    matrix[keys_map[key[0]]][keys_map[key[1]]] = J[key]

resp = solver.solve(matrix)
res = resp['QdeepHybridSolver']['configuration']

print("Energy: ", resp['QdeepHybridSolver']['energy'])

print("Parameters amount:", len(res))

total_distance = 0

for k in range(CarsNum):

    for i in range(ClientsNum):
        if res[keys_map[f"u{i}{k}"]] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: Depot -> {i}, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")

    for i in range(ClientsNum):
        for j in range(ClientsNum):
            if res[keys_map[f"x{i}{j}{k}"]] == 1:
                total_distance += Distances[i][j]
                print(f"Car {k}: {i} -> {j}, distance: {Distances[i][j]}")

    for i in range(ClientsNum):
        if res[keys_map[f"n{i}{k}"]] == 1:
            total_distance += Distances[i][ClientsNum + CarDepotMap[k]]
            print(f"Car {k}: {i} -> Depot, distance: {Distances[i][ClientsNum + CarDepotMap[k]]}")


print("Answer QDeep: ", total_distance)

Energy:  -334000010821632
Parameters amount: 202
Car 0: Depot -> 1, distance: 895
Car 0: 1 -> Depot, distance: 895
Car 1: Depot -> 3, distance: 3988
Car 1: 0 -> 0, distance: 0
Car 1: 3 -> 2, distance: 2151
Car 1: 3 -> 5, distance: 1123
Car 1: 5 -> 3, distance: 1123
Car 1: 2 -> Depot, distance: 327
Answer QDeep:  10502
